# Body Fat Data Analysis
#### Group members:Yan Chu, Boliang Ma, Jiacheng Xu

The goal is to come up with a simple, robust, accurate and precise  “rule-of-thumb” method to estimate percentage of body fat based on the data set.

The data set contains measurements from 252 men who had their body fat percentage accurately measured via underwater weighing.

## Step 1: Analyzing Raw Data

In [6]:
body_fit <- read.csv("https://github.com/JXU0728/STAT-628-module-1/blob/master/data/BodyFat.csv",header = T, quote = "")

In [7]:
dim(body_fit)

[1] 5510    1